# Generate Serializer Module

The Jervis Serializer Module is pretty big and contains a lot of classes. This notebook will generate the module from parsing the classpath and find all relevant sub-classes.

In [5]:
// implementation("org.jetbrains.kotlin:kotlin-reflect")
@file:DependsOn("io.github.classgraph:classgraph:4.8.157")

In [6]:
import kotlin.reflect.KClass

// Helper class to track the type hiearchy
class SerializedClass(
    val type: KClass<*>,
    val finalSubTypes: MutableList<KClass<*>> = mutableListOf(),
    val openSubTypes: MutableList<SerializedClass> = mutableListOf()
)

In [7]:
import com.jervisffb.engine.actions.CalculatedAction
import com.jervisffb.engine.actions.GameAction
import com.jervisffb.engine.model.inducements.BiasedReferee
import com.jervisffb.engine.model.inducements.Spell
import com.jervisffb.engine.model.inducements.wizards.Wizard
import com.jervisffb.engine.model.locations.Location
import com.jervisffb.engine.rules.Rules
import com.jervisffb.engine.rules.TeamActions
import com.jervisffb.engine.rules.bb2020.SkillSettings
import com.jervisffb.engine.rules.bb2020.procedures.DieRoll
import com.jervisffb.engine.rules.bb2020.skills.RerollSource
import com.jervisffb.engine.rules.bb2020.skills.Skill
import com.jervisffb.engine.rules.common.pathfinder.PathFinder
import com.jervisffb.engine.rules.common.roster.Position
import com.jervisffb.engine.rules.common.roster.Roster
import com.jervisffb.engine.serialize.SerializedPlayer
import com.jervisffb.engine.serialize.SerializedTeam
import io.github.classgraph.ClassGraph
import kotlin.reflect.KClass

// List of top-level interfaces and abstract classes that can be
// serialized
// Do not serialize Procedures and ProcedureContex's. They should
// be generated again when the Rules Engine run.
val topLevelClasses = mutableListOf<KClass<*>>(
    BiasedReferee::class,
    DieRoll::class,
    GameAction::class,
    PathFinder::class,
    Position::class,
    Roster::class,
    Rules::class,
    SerializedPlayer::class,
    SerializedTeam::class,
    SkillSettings::class,
    Spell::class,
    TeamActions::class,
    Wizard::class,
)

// Some subtypes we explicetely do not want to serialize.
// List them here.
val ignoredClasses: Set<KClass<*>> = setOf(
    CalculatedAction::class
)

// Create ClassGraph object we can use to iterate over the class hieachy.
val classGraph = ClassGraph()
    .enableAllInfo()
    .acceptPackages("com.jervisffb.engine")
    .enableInterClassDependencies()
    .scan()

fun visitOpenClass(clazz: KClass<*>, serializeInfo: SerializedClass) {
    val subClasses = if (clazz.java.isInterface) {
        classGraph
            .getClassesImplementing(clazz.java)
            .map { Class.forName(it.name).kotlin }
            .filterNot { it: KClass<*> -> ignoredClasses.contains(it) }
    } else {
        classGraph
            .getSubclasses(clazz.java)
            .map { Class.forName(it.name).kotlin }
            .filterNot { it: KClass<*> -> ignoredClasses.contains(it) }
    }
    for (subClass: KClass<*> in subClasses) {
        if (subClass.java.isInterface || subClass.isAbstract || subClass.isSealed) {
            val newTypeInfo = SerializedClass(subClass)
            serializeInfo.openSubTypes.add(newTypeInfo)
            visitOpenClass(subClass, newTypeInfo)
        } else {
            serializeInfo.finalSubTypes.add(subClass)
        }
    }
}

// Build up the class hiearchy
val serializableHierachy: List<SerializedClass> = topLevelClasses.map { serializableType: KClass<*> ->
    val info = SerializedClass(serializableType)
    visitOpenClass(serializableType, info)
    info
}

In [8]:

// Create the serializer module code
val code = buildString {
    appendLine("""
        package com.jervisffb.engine.serialize

        import kotlinx.serialization.modules.SerializersModule
        import kotlinx.serialization.modules.polymorphic
        import kotlinx.serialization.modules.subclass

        /**
         * WARNING: This file is autogenerated. Do not modify manully
         * Instead rerun the notebook in `tools/GenerateSerializers.ipynb` to update.
         */
        val generatedJervisSerializerModule = SerializersModule {
    """.trimIndent())

    // Helper for recursively printing polymorphic classes
    fun printPolymorphic(type: SerializedClass, indent: Int) {
        val topIndent = " ".repeat(indent*4)
        val nestedIndent = " ".repeat((indent + 1) * 4)
        appendLine("${topIndent}polymorphic(${type.type.qualifiedName}::class) {")
        for (subType in type.finalSubTypes) {
            appendLine("${nestedIndent}subclass(${subType.qualifiedName}::class)")
        }
        for (subType in type.openSubTypes) {
            printPolymorphic(subType, indent + 1)
        }
        appendLine("$topIndent}")
    }

    // Start by printing the top-level classes
    for (type: SerializedClass in serializableHierachy) {
        printPolymorphic(type, 1)
    }
    appendLine("}")
}

code

package com.jervisffb.engine.serialize

import kotlinx.serialization.modules.SerializersModule
import kotlinx.serialization.modules.polymorphic
import kotlinx.serialization.modules.subclass

/**
 * WARNING: This file is autogenerated. Do not modify manully
 * Instead rerun the notebook in `tools/GenerateSerializers.ipynb` to update.
 */
val generatedJervisSerializerModule = SerializersModule {
    polymorphic(com.jervisffb.engine.model.inducements.BiasedReferee::class) {
        subclass(com.jervisffb.engine.model.inducements.StandardBiasedReferee::class)
    }
    polymorphic(com.jervisffb.engine.rules.bb2020.procedures.DieRoll::class) {
        subclass(com.jervisffb.engine.rules.bb2020.procedures.BlockDieRoll::class)
        subclass(com.jervisffb.engine.rules.bb2020.procedures.D6DieRoll::class)
    }
    polymorphic(com.jervisffb.engine.actions.GameAction::class) {
        subclass(com.jervisffb.engine.actions.BlockTypeSelected::class)
        subclass(com.jervisffb.engine.actions.